# Purpose
This notebook describes beginning to a Springboard Project: Opportunity Pipeline Forecasting. We will be trying to understand the tables, columns and information flow. Typically we also look for data issues try for resolution. At the end of this activity, the data sources and their treatment is finalized. Code in this notebook will not be part of the production code.

# Initialization

In [70]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
import os
import os.path as op
import pandas as pd 
import great_expectations as ge
from dateutil.relativedelta import relativedelta
os.environ['TA_DEBUG'] = "False"
os.environ['TA_ALLOW_EXCEPTIONS'] = "True"

In [72]:
import warnings

warnings.filterwarnings('ignore', message="The sklearn.metrics.classification module", category=FutureWarning)
warnings.filterwarnings('ignore', message=".*title_format is deprecated. Please use title instead.*")
warnings.filterwarnings('ignore', message="The default value of regex will change from True to False in a future version.", 
                        category=FutureWarning)
warnings.filterwarnings('ignore', message="this method is deprecated in favour of `Styler.to_html()`")

In [73]:
%%time
from ta_lib.core.api import (
    create_context,
    get_package_path,
    display_as_tabs,
    initialize_environment,
    string_cleaning,
    setanalyse,
    merge_expectations
)
import ta_lib.core.api as dataset
import ta_lib.eda.api as analysis
import ta_lib.reports.api as health

CPU times: total: 0 ns
Wall time: 0 ns


In [74]:
# Initialization
initialize_environment(debug=False, hide_warnings=True)

# Data

## Background

The sales group of a technology and chip manufacturing company is provided with annual targets in \\$ that they need to sell. The team tries to achieve the same by pursuing multiple opportunities (an opportunity can be defined as a potential customer with specific asks of products in type, design and quantity with their \\$ values negotiated) across customers and business segments.
The sales enablement group of the same organization has identified that while a number of opportunities are being pursued, the sales group lacks a means to quantify the potential of conversion of these group of opportunities (hereon referred as sales pipeline portfolio) within a specific timeframe (say a quarter). The conversion potential of a sales pipeline portfolio is realized by,
 - Quality of an opportunity: How likely is an opportunity to convert within a specific timeframe? 


Below is an overview of each of the keys in the dataset:

| | Key | Dataset | Details |
|--|--|--|--|
| 1 | Opportunity ID | prod_data.csv & opp_data.csv | Opportunity Identifier |
| 2 | Product ID | prod_data.csv | Product Identifier |
| 3 | Product Segment Name | prod_data.csv | Product Segment |
| 4 | Product Status | prod_data.csv | Product Status (Pending,Win Approved,Win Submitted,Lost,Cancelled)  |
|5 | Product \\$ | prod_data.csv | Price in Proposal |
| 6 | Product Quantity | prod_data.csv | Quantity in Proposal | 
| 7 | Decision Date | prod_data.csv & opp_data | Opportunity Decision Date (Deadline to decide on opportunity)| 
| 8 | Snapshot Time | prod_data.csv | Quantity in Proposal | 
| 9 | Transition To Stage | opp_data.csv | Oppurtunity Stage at Snapshot Time | 
| 10 | Transition To Timestamp | opp_data.csv | Snapshot Time |
| 11 | Transition From Stage Name | opp_data.csv | Opportunity Stage at previous Snapshot | 
| 12 | Transition From Timestamp | opp_data.csv | Previous Snapshot Time| 
| 13 | Customer Name | opp_data.csv | Customer Name | 
| 14 | Risk Status | opp_data.csv | Opportunity Risk |
| 15 | Creation Date | opp_data.csv | Opportunity Creation Date |
| 16 | Opportunity Status | opp_data.csv | Opportuinity Status (Open, Closed won,Closed Lost) |
| 17 | Opportunity Type | opp_data.csv | Opportunity Type(Deal, if customer interested in existing product configuration and Design if customer interested in entirely new design) |
| 18 | Core Consumption Market | opp_data.csv | Consumption market(Entertainment,Office use etc. )|
| 19 | Core Product Segment | opp_data.csv | Product Segment(Processors, Graphic Card, Mother Board etc.) |
| 20 | Core Sales Segment | opp_data.csv | Sales Segment |
| 21 | Geography | opp_data.csv | Customer Geography |
| 22 | Core Product Application | opp_data.csv | Product Application(Pc,Server,Mobile,Tablet,etc.) |

In [75]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)
dataset.list_datasets(context)

['/raw/opportunity',
 '/raw/product',
 '/cleaned/opportunity',
 '/cleaned/product',
 '/processed/merged_final_dataset',
 '/train/features',
 '/train/target',
 '/test/features',
 '/test/target']

In [76]:
# Loading all datasets in a loop
data = dict()
for i in dataset.list_datasets(context):
    if '/raw/' in i:
        key_ = i.replace('/raw/','')+'_df'
        data[key_] = dataset.load_dataset(context,i)
        # Standardize column names
        data[key_].columns = string_cleaning(data[key_].columns,lower=True)

## Exploratory Data Analysis

### Shape of Data

In [77]:
(
    pd.DataFrame({x:data[x].shape for x in data.keys()})
    .T
    .rename(columns={0:'rows',1:'columns'})
    .sort_values('rows',ascending=False)
)

,rows,columns
opportunity_df,313571,16
product_df,142431,8


## opportunity_df

* transition to stage, transition from stage: convert it into int(stage 4 - 4)
* transition to timestamp, transition from timestamp: convert timestamp to datetime object
* core consumption market, core product segment. core sales segment, geography, core product application: make strings to integer(geo 1 - geo)

## product_df

* snapshot time: convert timestamp to datetime object
* product segment: make string to integer(product segment NA - NA)

## Variable summary

In [78]:
summaries = [analysis.get_variable_summary(data[x]) for x in data.keys()]
display_as_tabs([(x, summaries[idx]) for idx, x in enumerate(data.keys())])

Tabs
    [0] DataFrame(height=300, name='opportunity_df', sizing_mode='fixed', value=                          ..., width=800)
    [1] DataFrame(height=300, name='product_df', sizing_mode='fixed', value=                          ..., width=800)

## Health Analysis

Get an overview of the overall health of your dataset. This is usually quick to compute and hopefully highlights some problems to focus on.



In [79]:
summaries_and_plots = [analysis.get_data_health_summary(data[x], return_plot=True) for x in data.keys()]
plots = [x[1] for x in summaries_and_plots]
display_as_tabs([(x, plots[idx]) for idx, x in enumerate(data.keys())])

Tabs
    [0] HoloViews(Layout, name='opportunity_df')
    [1] HoloViews(Layout, name='product_df')

In [81]:
data['opportunity_df'].head()

,ïopportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,5,Stage 3,20150211130001,NaN,NaN,Customer 83,NaN,2015-02-11,2015-03-12,NaN,NaN,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
1,5,Stage 3,20150213050002,Stage 3,2.015021e+13,Customer 83,NaN,2015-02-11,2015-03-12,NaN,NaN,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
2,5,Stage 3,20150218210007,Stage 3,2.015021e+13,Customer 83,NaN,2015-02-11,2015-03-12,NaN,NaN,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
3,5,Stage 3,20150304210002,Stage 3,2.015022e+13,Customer 83,NaN,2015-02-11,2015-03-12,NaN,NaN,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
4,5,Stage 3,20150304210002,Stage 3,2.015030e+13,Customer 83,NaN,2015-02-11,2015-03-12,Open,NaN,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4


In [82]:
data['product_df'].head()

,ïopportunity id,product id,product segment name,product status,product $,product quantity,decision date,snapshot time
0,1361,6669,Product Segment NA,Win Approved,0.0,1.0,2015-08-12,20151125050002
1,1361,6669,Product Segment NA,Win Approved,0.0,1.0,2015-08-12,20151025050002
2,1361,6669,Product Segment NA,Win Approved,0.0,1.0,2015-08-12,20160105050003
3,1361,6669,Product Segment NA,Win Approved,0.0,1.0,2015-08-12,20160101050003
4,1361,6669,Product Segment NA,Pending,0.0,1.0,2015-08-31,20150525050003


## Data Cleaning

In [83]:
data={}
for i in dataset.list_datasets(context):
    if '/raw/' in i:
        dataset_name = i.replace('/raw/','')
        key_ = dataset_name+'_df'
        data[key_] = dataset.load_dataset(context,i)
        
        # converting column names to lowercase for uniformity
        data[key_].columns = string_cleaning(data[key_].columns,lower=True)
        
        if key_ == 'opportunity_df':
            
            data['opportunity_df'].rename(columns = {'ïopportunity id' : 'opportunity id'}, inplace = True)
            
            #calculating time taken for transition
            data['opportunity_df']['time taken for transition']=data['opportunity_df']['transition to timestamp']-data['opportunity_df']['transition from timestamp']
            
            # Convert transition timestamps to datetime objects
            data[key_]['transition from timestamp']=pd.to_datetime(data[key_]['transition from timestamp'], format='%Y%m%d%H%M%S')
            data[key_]['transition to timestamp']=pd.to_datetime(data[key_]['transition to timestamp'], format='%Y%m%d%H%M%S')
            

            # Encode 'transition to stage', 'transition from stage name', 'customer name', 'risk status', 
            # 'opportunity status', and 'opportunity type' as numerical values
            for col in ['transition to stage', 'transition from stage name']:
                temp=[]
                for i in data[key_][col]:
                    if type(i)==str:
                        temp.append(int(i[-1]))
                    else:
                        temp.append(-1)           
                data[key_][col]=temp 
            
            temp=[]
            for i in data[key_]["customer name"]:
                if i!='Customer NA':
                    cust_no = int(i.split(" ")[-1])
                    temp.append(cust_no)
                else:
                    temp.append(-1)           
            data[key_]["customer name"]=temp
            
            for col in ['risk status', 'opportunity status', 'opportunity type']:
                unique_vals = data[key_][col].unique()
                val_dict = {unique_vals[i]: i for i in range(len(unique_vals))}
                temp = []
                for val in data[key_][col]:
                    temp.append(val_dict[val])
                data[key_][col] = temp
            
            # Encode 'core consumption market' and 'core product segment' as numerical values
            for col in ['core consumption market', 'core product segment','core sales segment']:
                temp = []
                for val in data[key_][col]:
                    if val != 'Customer NA':
                        num = int(val.split(" ")[-1])
                        temp.append(num)
                    else:
                        temp.append(-1)
                data[key_][col] = temp
            
            temp=[]
            for i in data[key_]["geography"]:
                if i!='Geo NA':
                    geo_no = int(i.split(" ")[-1])
                    temp.append(geo_no)
                else:
                    temp.append(-1)           
            data[key_]["geography"]=temp
            
            temp=[]
            for i in data[key_]["core product application"]:
                if i!='Prd App 4 NA':
                    prod_app_no = int(i.split(" ")[-1])
                    temp.append(prod_app_no)
                else:
                    temp.append(-1)           
            data[key_]["core product application"]=temp
            
            opportunity = data['opportunity_df']
            
            #removing duplicates
            data['opportunity_df']=data['opportunity_df'].drop_duplicates(subset=["opportunity id","transition to stage","transition from stage name","customer name",
            "risk status","creation date","decision date","opportunity status","opportunity type","core consumption market","core sales segment","geography",
            "core product application"],keep="first")

            data['opportunity_df'].fillna(value='0',inplace=True)

        if key_ == 'product_df':
            
            data['product_df'].rename(columns = {'ïopportunity id' : 'opportunity id'}, inplace = True)
            
            #calculating time taken for transition
            data[key_]['snapshot time']=pd.to_datetime(data[key_]['snapshot time'], format='%Y%m%d%H%M%S')
            
            
            temp=[]
            prod_status=data[key_]['product status'].unique()
            prod_stat_dict= {prod_status[i] : i for i in range(len(prod_status)) }
            for i in data[key_]["product status"]:
                temp.append(prod_stat_dict[i])         
            data[key_]["product status"]=temp
            
            #sorting in ascending order
            data['product_df'].sort_values(by=['opportunity id','product id','snapshot time','decision date','product $','product quantity'],inplace=True)

            #dropping product segment name as it has only NA value
            data['product_df'].drop(columns=['product segment name'],inplace=True)
            product=data['product_df']
            #dropping duplicates
            data['product_df'].drop_duplicates(subset=['opportunity id','product id','product status','product $','product quantity','product quantity'],inplace=True)
            data['product_df']
        
        dataset.save_dataset(context, data[key_], 'cleaned/'+dataset_name, index = False)

In [84]:
opportunity = dataset.load_dataset(context, 'cleaned/opportunity')
product = dataset.load_dataset(context, 'cleaned/product')

In [85]:
opportunity.drop(['Unnamed: 0'], axis =1, inplace = True)

In [86]:
product.drop(['Unnamed: 0'], axis =1, inplace = True)

In [87]:
product.head()

,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time
0,5,404,1,230400.0,1200.0,12-03-2015,2015-02-11 21:00:01
1,5,404,0,230400.0,1200.0,16-03-2015,2015-03-16 13:00:01
2,19,16377,1,0.0,400000.0,14-08-2015,2015-05-11 05:00:04
3,19,16377,0,0.0,400000.0,18-08-2015,2015-08-18 13:00:02
4,19,16378,1,0.0,200000.0,14-08-2015,2015-05-11 05:00:04


In [106]:
merged_df=pd.merge(opportunity,product,how='inner',left_on=['opportunity id','decision date'],right_on=['opportunity id','decision date'])
dataset.save_dataset(context,merged_df , 'processed/'+'merged_final_dataset')

In [107]:
merged_df

,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,...,core product segment,core sales segment,geography,core product application,time taken for transition,product id,product status,product $,product quantity,snapshot time
0,5,3,2015-02-11 13:00:01,-1,0,83,0,11-02-2015,12-03-2015,0,...,3,8,-1,4,0.0,404,1,230400.0,1200.0,2015-02-11 21:00:01
1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,0,11-02-2015,12-03-2015,0,...,3,8,-1,4,1920001.0,404,1,230400.0,1200.0,2015-02-11 21:00:01
2,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,0,11-02-2015,12-03-2015,1,...,3,8,-1,4,0.0,404,1,230400.0,1200.0,2015-02-11 21:00:01
3,5,4,2015-03-16 13:00:01,3,2015-03-04 21:00:02,83,0,11-02-2015,12-03-2015,2,...,3,8,-1,4,11919999.0,404,1,230400.0,1200.0,2015-02-11 21:00:01
4,5,4,2015-03-17 13:00:01,4,2015-03-16 13:00:01,83,0,11-02-2015,12-03-2015,2,...,3,8,1,4,1000000.0,404,1,230400.0,1200.0,2015-02-11 21:00:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24310,42789,1,2016-06-12 21:00:04,6,2016-06-10 07:13:44,9787,0,15-11-2015,01-09-2016,3,...,5,4,2,5,2138660.0,38158,1,6000.0,1000.0,2015-11-16 13:00:03
24311,42789,1,2016-06-15 13:00:03,1,2016-06-12 21:00:04,9787,0,15-11-2015,01-09-2016,3,...,5,4,2,5,2919999.0,38158,1,60000.0,10000.0,2015-11-15 13:00:03
24312,42789,1,2016-06-15 13:00:03,1,2016-06-12 21:00:04,9787,0,15-11-2015,01-09-2016,3,...,5,4,2,5,2919999.0,38158,1,6000.0,1000.0,2015-11-16 13:00:03
24313,42789,1,2016-07-01 05:00:04,1,2016-06-25 05:00:03,9787,1,15-11-2015,01-09-2016,3,...,5,4,2,5,76000001.0,38158,1,60000.0,10000.0,2015-11-15 13:00:03


In [108]:
# 2 : win
# 1 : open
# 0 : na
# 3 : lost



In [111]:
drop_index = merged_df[(merged_df['opportunity status'] ==0)].index
merged_df.drop(drop_index, inplace = True)

In [112]:
merged_df

,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,...,core product segment,core sales segment,geography,core product application,time taken for transition,product id,product status,product $,product quantity,snapshot time
2,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,0,11-02-2015,12-03-2015,1,...,3,8,-1,4,0.0,404,1,230400.0,1200.0,2015-02-11 21:00:01
3,5,4,2015-03-16 13:00:01,3,2015-03-04 21:00:02,83,0,11-02-2015,12-03-2015,2,...,3,8,-1,4,11919999.0,404,1,230400.0,1200.0,2015-02-11 21:00:01
4,5,4,2015-03-17 13:00:01,4,2015-03-16 13:00:01,83,0,11-02-2015,12-03-2015,2,...,3,8,1,4,1000000.0,404,1,230400.0,1200.0,2015-02-11 21:00:01
5,5,4,2015-03-23 21:00:01,4,2015-03-17 13:00:01,83,0,11-02-2015,12-03-2015,2,...,3,8,3,4,6080000.0,404,1,230400.0,1200.0,2015-02-11 21:00:01
6,5,4,2015-08-10 18:37:57,4,2015-07-18 05:00:03,83,0,11-02-2015,12-03-2015,2,...,3,8,3,4,92133754.0,404,1,230400.0,1200.0,2015-02-11 21:00:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24310,42789,1,2016-06-12 21:00:04,6,2016-06-10 07:13:44,9787,0,15-11-2015,01-09-2016,3,...,5,4,2,5,2138660.0,38158,1,6000.0,1000.0,2015-11-16 13:00:03
24311,42789,1,2016-06-15 13:00:03,1,2016-06-12 21:00:04,9787,0,15-11-2015,01-09-2016,3,...,5,4,2,5,2919999.0,38158,1,60000.0,10000.0,2015-11-15 13:00:03
24312,42789,1,2016-06-15 13:00:03,1,2016-06-12 21:00:04,9787,0,15-11-2015,01-09-2016,3,...,5,4,2,5,2919999.0,38158,1,6000.0,1000.0,2015-11-16 13:00:03
24313,42789,1,2016-07-01 05:00:04,1,2016-06-25 05:00:03,9787,1,15-11-2015,01-09-2016,3,...,5,4,2,5,76000001.0,38158,1,60000.0,10000.0,2015-11-15 13:00:03


In [113]:
merged_df['opportunity status'].replace(3,0,inplace=True)
merged_df['opportunity status'].replace(1,0,inplace=True)
merged_df['opportunity status'].replace(2,1,inplace=True)

merged_df[(merged_df['product status']==0) & (merged_df['opportunity status']==0)]

,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,...,core product segment,core sales segment,geography,core product application,time taken for transition,product id,product status,product $,product quantity,snapshot time
71,46,3,2015-06-08 13:00:03,3,2015-06-03 21:00:03,108,0,11-02-2015,08-06-2015,0,...,3,1,3,4,4920000.0,15983,0,1056000.0,3000.0,2015-06-12 21:00:09
73,46,3,2015-06-08 13:00:03,3,2015-06-03 21:00:03,108,0,11-02-2015,08-06-2015,0,...,3,1,3,4,4920000.0,15984,0,777000.0,3000.0,2015-06-12 21:00:09
112,99,3,2015-08-10 18:37:57,2,2015-07-18 05:00:03,108,0,11-02-2015,10-08-2015,0,...,3,1,3,4,92133754.0,16980,0,2000000.0,20000.0,2015-08-24 21:00:05
114,99,3,2015-08-10 18:37:57,2,2015-07-18 05:00:03,108,0,11-02-2015,10-08-2015,0,...,3,1,3,4,92133754.0,16981,0,15000000.0,60000.0,2015-08-24 21:00:05
177,151,5,2015-03-04 21:00:02,5,2015-03-04 21:00:02,1127,0,11-02-2015,23-03-2015,0,...,3,8,-1,22,0.0,231,0,15400.0,700.0,2015-03-25 05:00:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23883,28191,5,2015-12-23 21:00:04,-1,0,3340,0,24-12-2015,23-12-2015,0,...,3,8,3,4,0.0,45159,0,2002.0,11.0,2015-12-24 13:00:03
23909,28196,5,2015-12-24 05:00:02,-1,0,8044,0,24-12-2015,23-12-2015,0,...,3,8,3,22,0.0,45233,0,90.0,3.0,2015-12-24 13:00:03
23911,28196,5,2015-12-24 05:00:02,-1,0,8044,0,24-12-2015,23-12-2015,0,...,3,8,3,22,0.0,45236,0,107.0,1.0,2015-12-24 13:00:03
23925,28199,5,2015-12-24 05:00:02,-1,0,8077,0,24-12-2015,23-12-2015,0,...,3,8,3,22,0.0,45253,0,562.0,2.0,2015-12-24 13:00:03


In [114]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23343 entries, 2 to 24314
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   opportunity id              23343 non-null  int64  
 1   transition to stage         23343 non-null  int64  
 2   transition to timestamp     23343 non-null  object 
 3   transition from stage name  23343 non-null  int64  
 4   transition from timestamp   23343 non-null  object 
 5   customer name               23343 non-null  int64  
 6   risk status                 23343 non-null  int64  
 7   creation date               23343 non-null  object 
 8   decision date               23343 non-null  object 
 9   opportunity status          23343 non-null  int64  
 10  opportunity type            23343 non-null  int64  
 11  core consumption market     23343 non-null  int64  
 12  core product segment        23343 non-null  int64  
 13  core sales segment          233

In [115]:
merged_df.isnull().sum()

opportunity id                0
transition to stage           0
transition to timestamp       0
transition from stage name    0
transition from timestamp     0
customer name                 0
risk status                   0
creation date                 0
decision date                 0
opportunity status            0
opportunity type              0
core consumption market       0
core product segment          0
core sales segment            0
geography                     0
core product application      0
time taken for transition     0
product id                    0
product status                0
product $                     0
product quantity              0
snapshot time                 0
dtype: int64

In [116]:
merged_df['cost_per_unit']=merged_df['product $']/merged_df['product quantity']
merged_df['cost_per_unit'].fillna(value=0,inplace=True)

In [117]:
dataset.save_dataset(context,merged_df , 'processed/'+'merged_final_dataset')

In [118]:
merged_df

,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,...,core sales segment,geography,core product application,time taken for transition,product id,product status,product $,product quantity,snapshot time,cost_per_unit
2,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,0,11-02-2015,12-03-2015,0,...,8,-1,4,0.0,404,1,230400.0,1200.0,2015-02-11 21:00:01,192.0
3,5,4,2015-03-16 13:00:01,3,2015-03-04 21:00:02,83,0,11-02-2015,12-03-2015,1,...,8,-1,4,11919999.0,404,1,230400.0,1200.0,2015-02-11 21:00:01,192.0
4,5,4,2015-03-17 13:00:01,4,2015-03-16 13:00:01,83,0,11-02-2015,12-03-2015,1,...,8,1,4,1000000.0,404,1,230400.0,1200.0,2015-02-11 21:00:01,192.0
5,5,4,2015-03-23 21:00:01,4,2015-03-17 13:00:01,83,0,11-02-2015,12-03-2015,1,...,8,3,4,6080000.0,404,1,230400.0,1200.0,2015-02-11 21:00:01,192.0
6,5,4,2015-08-10 18:37:57,4,2015-07-18 05:00:03,83,0,11-02-2015,12-03-2015,1,...,8,3,4,92133754.0,404,1,230400.0,1200.0,2015-02-11 21:00:01,192.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24310,42789,1,2016-06-12 21:00:04,6,2016-06-10 07:13:44,9787,0,15-11-2015,01-09-2016,0,...,4,2,5,2138660.0,38158,1,6000.0,1000.0,2015-11-16 13:00:03,6.0
24311,42789,1,2016-06-15 13:00:03,1,2016-06-12 21:00:04,9787,0,15-11-2015,01-09-2016,0,...,4,2,5,2919999.0,38158,1,60000.0,10000.0,2015-11-15 13:00:03,6.0
24312,42789,1,2016-06-15 13:00:03,1,2016-06-12 21:00:04,9787,0,15-11-2015,01-09-2016,0,...,4,2,5,2919999.0,38158,1,6000.0,1000.0,2015-11-16 13:00:03,6.0
24313,42789,1,2016-07-01 05:00:04,1,2016-06-25 05:00:03,9787,1,15-11-2015,01-09-2016,0,...,4,2,5,76000001.0,38158,1,60000.0,10000.0,2015-11-15 13:00:03,6.0
